In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('./data/train.csv')
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [3]:
df.set_index('id', inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57477 entries, 30192 to 4294947231
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model_a         57477 non-null  object
 1   model_b         57477 non-null  object
 2   prompt          57477 non-null  object
 3   response_a      57477 non-null  object
 4   response_b      57477 non-null  object
 5   winner_model_a  57477 non-null  int64 
 6   winner_model_b  57477 non-null  int64 
 7   winner_tie      57477 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 3.9+ MB


#### Unification of winner columns


In [5]:
df['winner'] = df.apply(lambda row: 1 if row['winner_model_a'] == 1 else (2 if row['winner_model_b'] == 1 else 0), axis=1)
df.drop(['winner_model_a', 'winner_model_b', 'winner_tie'], axis=1, inplace=True)
df.head()

,model_a,model_b,prompt,response_a,response_b,winner
id,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1
53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2
65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0
96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1
198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2


In [6]:
len(pd.unique(df['model_a']))


64

In [7]:
longest_str = lambda column: column.apply(len).max()
print(longest_str(df['prompt']))
print(longest_str(df['model_a']))
print(longest_str(df['model_b']))




33056
30
30


In [8]:
df.describe()

,winner
count,57477.000000
mean,1.032900
std,0.806133
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,2.000000


In [9]:
dummies = pd.get_dummies(df, columns=['model_a', 'model_b'], prefix=['model_a', 'model_b'], dtype=int)
df.drop(['model_a', 'model_b'], axis=1, inplace=True)
df = pd.concat([dummies, df])
df.head()

,prompt,response_a,response_b,winner,model_a_RWKV-4-Raven-14B,model_a_alpaca-13b,model_a_chatglm-6b,model_a_chatglm2-6b,model_a_chatglm3-6b,model_a_claude-1,...,model_b_stripedhyena-nous-7b,model_b_tulu-2-dpo-70b,model_b_vicuna-13b,model_b_vicuna-33b,model_b_vicuna-7b,model_b_wizardlm-13b,model_b_wizardlm-70b,model_b_yi-34b-chat,model_b_zephyr-7b-alpha,model_b_zephyr-7b-beta
id,,,,,,,,,,,,,,,,,,,,,
30192,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53567,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65089,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96401,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198779,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
import tensorflow_hub as hub

sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def generate_embeddings(text_column):
    embeddings = sentence_encoder(text_column.tolist())
    return np.array(embeddings)

df['prompt'] = generate_embeddings(df['prompt'])
df['model_a'] = generate_embeddings(df['model_a'])
df['model_b'] = generate_embeddings(df['model_b'])


KeyError: 'model_a'

In [ ]:
df.head()